네이버와 다음 카페 특정 게시판 게시물의 고유번호, 제목, 게시일자, 조회수를 크롤링합니다.
개별 카페 특성에 따라 정상적으로 작동하지 않을 수 있습니다.

In [1]:
import re
import json
import math
import datetime
import requests
import urllib.request
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from numpy import nan as NA
from pandas import Series,DataFrame
from selenium import webdriver
import time

In [5]:
def get_naver_title(domain,clubid,menuid) :

    c_no=[]
    cate=[]
    title=[]
    time=[]
    m_no=[]
    
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    driver = webdriver.Chrome('C:\chromedriver', options=options)

    for i in range(2,10) :

        driver.get('https://cafe.naver.com/'+str(domain)+'?iframe_url=/ArticleList.nhn%3Fsearch.clubid='+str(clubid)+'%26search.menuid='+str(menuid)+'%26userDisplay=50%26search.boardtype=L%26search.page='+str(i))
        driver.implicitly_wait(1)
        driver.switch_to.frame('cafe_main')
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')

        for td_num in range(50) :
            c_no.append(soup.select('td.td_article')[td_num].select('div')[0].text.strip())
            cate.append(str(domain))
            try :
                soup.select('td.td_article')[td_num].select('div>a>span')[0].replaceWith('')
                soup.select('td.td_article')[td_num].select('div>a')[0].text.strip()
            except :
                soup.select('td.td_article')[td_num].select('div>a')[0].text.strip()
            title.append(soup.select('td.td_article')[td_num].select('div>a')[0].text.strip())
            time.append(soup.select('td.td_date')[td_num].text.strip())
            m_no.append(soup.select('td.td_view')[td_num].text.strip())
    driver.close()

    naver_title_list = pd.DataFrame({'no' : c_no, 'cate' : cate, 'title' : title, 'time' : time, 'm_no' : m_no})
    naver_title_list.time = naver_title_list.time.map(lambda x : pd.to_datetime(x,format="%Y.%m.%d.") if len(x) > 10 else pd.to_datetime(datetime.datetime.now().strftime('%Y.%m.%d.')))
    naver_title_list = naver_title_list.set_index('time')
    print(str(domain)+' : '+str(len(naver_title_list))+'개 추출')
    return naver_title_list

In [6]:
goda = {"domain" : 'ilovecat', "clubid" : 10070043,"menuid" :8}
goda_df=get_naver_title(**goda)

ilovecat : 400개 추출


In [7]:
goda_df

,no,cate,title,m_no
time,,,,
2019-09-15,5870046,ilovecat,후드형 화장실 쓰시는분들 질문요,94
2019-09-15,5870045,ilovecat,대치동학원가 임신한 길냥이 구조이야기~,150
2019-09-15,5870044,ilovecat,인천사시는 집사님들~~,91
2019-09-15,5870042,ilovecat,반스 신발 모델명 좀 알려주세요 ㅠㅠ!,197
2019-09-15,5870039,ilovecat,아까 도인같은 의사쌤 만났다고 글 올렸는데 강사모 카페 가보니까 희대의 사기꾼이었네...,625
2019-09-15,5870038,ilovecat,2개월된 강아지 각종학대 유기 사망사건 서명 부탁드립니다,22
2019-09-15,5870037,ilovecat,크는 속도가 엄청나네요,602
2019-09-15,5870036,ilovecat,동물학대에 쓰일 수 있는 화살촉 판매규제 서명 부탁드려요!,30
2019-09-15,5870035,ilovecat,펫샵 국내애견 운영실태 전수조사 서명 부탁드립니다,18


In [8]:
def get_daum_title_girls_gen() :
    
    c_no=[]
    cate=[]
    title=[]
    time=[]
    m_no=[]
    counter=[]
    
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")

    driver = webdriver.Chrome('C:\chromedriver', options=options)
    driver.get('http://cafe.daum.net/_c21_/bbs_list?grpid=1IHuH&fldid=UWV0&listnum=50')
    driver.implicitly_wait(1)
    driver.switch_to.frame('down')
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    for td_num in range(len(soup.select('td.num'))) :
        counter.append(soup.select('td.num')[td_num].text.strip())

    for td_num in range(counter.count(''),len(soup.select('td.num'))) :
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        c_no.append(soup.select('td.num')[td_num].text.strip())
        cate.append('girls_gen')
        try :
            soup.select('td.subject')[td_num].select('a.txt_point')[0].replaceWith('')
            title.append(soup.select('td.subject')[td_num].text.strip())
        except :
            title.append(soup.select('td.subject')[td_num].text.strip())
        time.append(soup.select('td.date')[td_num].text.strip())
        m_no.append(soup.select('td.count')[td_num].text.strip())

    for i in range(2,8) :
        driver.find_element_by_xpath('//*[@id="primaryContent"]/table/tbody/tr[2]/td[2]/div[2]/div/a['+str(i)+']').click()
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')

        for td_num in range(counter.count(''),len(soup.select('td.num'))) :
            c_no.append(soup.select('td.num')[td_num].text.strip())
            cate.append('girls_gen')
            try :
                soup.select('td.subject')[td_num].select('a.txt_point')[0].replaceWith('')
                title.append(soup.select('td.subject')[td_num].text.strip())
            except :
                title.append(soup.select('td.subject')[td_num].text.strip())
            time.append(soup.select('td.date')[td_num].text.strip())
            m_no.append(soup.select('td.count')[td_num].text.strip())

    for i in range(8,10) :
        driver.find_element_by_xpath('//*[@id="primaryContent"]/table/tbody/tr[2]/td[2]/div[2]/div/a[7]').click()
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')

        for td_num in range(counter.count(''),len(soup.select('td.num'))) :
            c_no.append(soup.select('td.num')[td_num].text.strip())
            cate.append('girls_gen')
            try :
                soup.select('td.subject')[td_num].select('a.txt_point')[0].replaceWith('')
                title.append(soup.select('td.subject')[td_num].text.strip())
            except :
                title.append(soup.select('td.subject')[td_num].text.strip())
            time.append(soup.select('td.date')[td_num].text.strip())
            m_no.append(soup.select('td.count')[td_num].text.strip())

    driver.close()

    daum_title_list = pd.DataFrame({'no' : c_no, 'cate' : cate, 'title' : title, 'time' : time, 'm_no' : m_no})
    daum_title_list.time = daum_title_list.time.map(lambda x : pd.to_datetime(x,format="%y.%m.%d") if len(x) > 7 else pd.to_datetime(datetime.datetime.now().strftime('%y.%m.%d'),yearfirst=True))
    daum_title_list = daum_title_list.set_index('time')
    return daum_title_list

In [9]:
daum_gg_df=get_daum_title_girls_gen()

In [10]:
daum_gg_df

,no,cate,title,m_no
time,,,,
2019-09-15,751870,girls_gen,[고양이] 발톱이 이상해..!!,9
2019-09-15,751869,girls_gen,[고양이] 원래 많이 자는거지??,2
2019-09-15,751868,girls_gen,[고양이] 댕댕주인인데 냥집사님들께 질문있읍니다..,16
2019-09-15,751867,girls_gen,[강아지] 잡아먹을까~~~,28
2019-09-15,751866,girls_gen,[고양이] 화장실 큰걸로 바꿔주려고 하거든,3
2019-09-15,751865,girls_gen,[사담]유모차 세탁 맡겨본여시 있어?,6
2019-09-15,751864,girls_gen,[고양이]애드보킷 직구햇는데 원래동물병원에서 살땐 주황색이엿는데,29
2019-09-15,751863,girls_gen,[고양이] 애기가 너무 말랐는데 로열캐닌 키튼 줘도 될까?,14
2019-09-15,751862,girls_gen,[강아지] 산책 원래 짧게 해도 돼..?,71


예시와 같이 추출할 수 있습니다. 순서대로 네이버 카페 '고양이라서 다행이다', 다음 카페 '여성시대'의 크롤링 예시입니다.
게시물의 내용이나 카페 자체가 비공개라 로그인이 필요한 경우 로그인 과정은 별도로 구현해야 합니다.(가능하지만 어렵습니다.)
네이버나 다음 카페는 iframe이 적용되어 있기 때문에, 소스코드 수정시 웹에 대한 이해도 선행되어야 합니다.